#### подобрать оптимальные параметры для модели
- regularization, iterations
- factors
- Вес (элемент в матрице user-item)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools
import copy
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src.metrics import precision_at_k, recall_at_k

In [2]:
def get_recommendations(user, model, sparse_user_item, N=5):
    res = [id_to_itemid[rec] for rec in 
            model.recommend(userid=userid_to_id[user],  
                            user_items=sparse_user_item[userid_to_id[user]],
                            N=N, 
                            filter_already_liked_items=False, 
                            filter_items=[itemid_to_id[999999]], 
                            recalculate_user=True)[0]]
    return res

In [3]:
data = pd.read_csv('../raw_data/retail_train.csv')

test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [4]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [5]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [6]:
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', 
                                  columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float)
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

C:\Users\013\AppData\Roaming\Python\Python38\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

- лучший вариант взвешивания

In [8]:
tfidf = tfidf_weight(user_item_matrix).tocsr()  
tfidfT = tfidf_weight(user_item_matrix.T).T.tocsr()
bm25 = bm25_weight(user_item_matrix).tocsr()
bm25T = bm25_weight(user_item_matrix.T).T.tocsr()

In [81]:
matrices = {'tfidf': tfidf, 'tfidfT': tfidfT, 'bm25': bm25, 'bm25T': bm25T}

In [82]:
model = AlternatingLeastSquares(
                                factors=100,  
                                regularization=0.02,
                                iterations=15,  
                                calculate_training_loss=True, 
                                num_threads=4,
                                random_state=42
                               )

In [87]:
for matrix in matrices:
    model.fit(matrices[matrix], show_progress=True)
    result[f'als_{matrix}'] = result['user_id'].apply(lambda x: get_recommendations(x, model, sparse_user_item, 5))
    prec5 = result.apply(lambda row: precision_at_k(row[f'als_{matrix}'], row['actual']), axis=1).mean()
    print(matrix, prec5)

  0%|          | 0/15 [00:00<?, ?it/s]

tfidf 0.23898139079333666


  0%|          | 0/15 [00:00<?, ?it/s]

tfidfT 0.1891283055827594


  0%|          | 0/15 [00:00<?, ?it/s]

bm25 0.23124387855043754


  0%|          | 0/15 [00:00<?, ?it/s]

bm25T 0.22409402546522733


Лучший по точности вариант матрицы для обучения - tfidf, немного отстаёт - bm25

- поиск по сетке

In [9]:
def grid_search(base_model, fit_matrix, predict_matrix, result_df, param_grid):
    model_params = []
    metrics = []
    keys, values = zip(*param_grid.items())
    
    for i, v in enumerate(itertools.product(*values)):
        params = dict(zip(keys, v))
        model_params.append(params)
        
        this_model = copy.deepcopy(base_model)
        for k, v in params.items():
            setattr(this_model, k, v)

        this_model.fit(fit_matrix, show_progress=True)
        
        result_df[f'als_{i}'] = result_df['user_id'].apply(lambda x: get_recommendations(x, this_model, predict_matrix, 5))
        prec5 = result_df.apply(lambda row: precision_at_k(row[f'als_{i}'], row['actual']), axis=1).mean()
        metrics.append(prec5)
        
        print(f'params={params}, precision@k={prec5}')
    
    return model_params, metrics

In [10]:
param_grid = {'factors': [25, 50, 100, 150, 300],
              'regularization': [0.02, 0.06, 0.1, 0.8],
              'iterations': [8, 12, 20, 30, 100]}


model = AlternatingLeastSquares(
                                calculate_training_loss=True, 
                                num_threads=4,
                                random_state=42
                               )

D:\programs\anaconda3\lib\site-packages\implicit\utils.py:28: UserWarning: OpenBLAS detected. Its highly recommend to set the environment variable 'export OPENBLAS_NUM_THREADS=1' to disable its internal multithreading
  warnings.warn(


In [12]:
%%time  # 100 моделей ~31 мин

par, metr = grid_search(base_model=model, 
                        fit_matrix=tfidf, 
                        predict_matrix=sparse_user_item, 
                        result_df=result, 
                        param_grid=param_grid)

  0%|          | 0/8 [00:00<?, ?it/s]

params={'factors': 25, 'regularization': 0.02, 'iterations': 8}, precision@k=0.23222331047991893


  0%|          | 0/12 [00:00<?, ?it/s]

params={'factors': 25, 'regularization': 0.02, 'iterations': 12}, precision@k=0.2308521057786456


  0%|          | 0/20 [00:00<?, ?it/s]

...

params={'factors': 300, 'regularization': 0.8, 'iterations': 20}, precision@k=0.20195886385895911


  0%|          | 0/30 [00:00<?, ?it/s]

params={'factors': 300, 'regularization': 0.8, 'iterations': 30}, precision@k=0.2000979431929457


  0%|          | 0/100 [00:00<?, ?it/s]

params={'factors': 300, 'regularization': 0.8, 'iterations': 100}, precision@k=0.197257590597451
Wall time: 31min 28s


In [14]:
# лучшие параметры
for param, metric in zip(par, metr):
    if metric > 0.242:
        print(param, metric)

{'factors': 50, 'regularization': 0.02, 'iterations': 30} 0.24250734573946808
{'factors': 50, 'regularization': 0.06, 'iterations': 30} 0.242605288932416
{'factors': 50, 'regularization': 0.1, 'iterations': 30} 0.24240940254651988
{'factors': 50, 'regularization': 0.8, 'iterations': 20} 0.24211557296767564
{'factors': 100, 'regularization': 0.02, 'iterations': 20} 0.24387855044074133
{'factors': 100, 'regularization': 0.06, 'iterations': 20} 0.24289911851126045
{'factors': 100, 'regularization': 0.1, 'iterations': 8} 0.242213516160624
{'factors': 100, 'regularization': 0.1, 'iterations': 20} 0.24280117531831233


{'factors': 50, 'regularization': 0.02, 'iterations': 30} 0.24250734573946808

{'factors': 50, 'regularization': 0.06, 'iterations': 30} 0.242605288932416

{'factors': 50, 'regularization': 0.1, 'iterations': 30} 0.24240940254651988

{'factors': 50, 'regularization': 0.8, 'iterations': 20} 0.24211557296767564

{'factors': 100, 'regularization': 0.02, 'iterations': 20} 0.24387855044074133

{'factors': 100, 'regularization': 0.06, 'iterations': 20} 0.24289911851126045

{'factors': 100, 'regularization': 0.1, 'iterations': 8} 0.242213516160624

{'factors': 100, 'regularization': 0.1, 'iterations': 20} 0.24280117531831233


- более тонкий поиск по сетке

In [36]:
param_grid = {'factors': [99, 100],
              'regularization': [0.02, 0.022],
              'iterations': [18, 20, 22]}


model = AlternatingLeastSquares(
                                calculate_training_loss=True, 
                                num_threads=4,
                                random_state=42
                               )

In [37]:
par1, metr1 = grid_search(base_model=model, 
                          fit_matrix=tfidf, 
                          predict_matrix=sparse_user_item, 
                          result_df=result, 
                          param_grid=param_grid)

  0%|          | 0/18 [00:00<?, ?it/s]

params={'factors': 99, 'regularization': 0.02, 'iterations': 18}, precision@k=0.24162585700293518


  0%|          | 0/20 [00:00<?, ?it/s]

params={'factors': 99, 'regularization': 0.02, 'iterations': 20}, precision@k=0.24152791380998717


  0%|          | 0/22 [00:00<?, ?it/s]

...

params={'factors': 100, 'regularization': 0.022, 'iterations': 18}, precision@k=0.24240940254652002


  0%|          | 0/20 [00:00<?, ?it/s]

params={'factors': 100, 'regularization': 0.022, 'iterations': 20}, precision@k=0.24338883447600088


  0%|          | 0/22 [00:00<?, ?it/s]

params={'factors': 100, 'regularization': 0.022, 'iterations': 22}, precision@k=0.24250734573946794


И так далее с разными параметрами - не улучшает результата.

Лучшие модель и метрика:

In [38]:
model = AlternatingLeastSquares(
                                factors=100,  
                                regularization=0.02,
                                iterations=20,  
                                calculate_training_loss=True, 
                                num_threads=4,
                                random_state=42
                               )

In [39]:
model.fit(tfidf, show_progress=True)
result[f'als_best_param'] = result['user_id'].apply(lambda x: get_recommendations(x, model, sparse_user_item, 5))
prec5 = result.apply(lambda row: precision_at_k(row[f'als_best_param'], row['actual']), axis=1).mean()
prec5

  0%|          | 0/20 [00:00<?, ?it/s]

0.24387855044074133